<i>STATUS: Draft<i>

In [13]:
import numpy as np
import sympy as sp
from IPython.display import HTML, IFrame
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Rectangle
from IPython.display import Image
import sys
import HTM_Code as hc
import pandas as pd
import calendar

In the last notebook, we built our first encoder. Admittedly, it was very simple. It just allows us to create integers and provide them to, and provide back indice location. It provided a little extra functionality so it could track previous, and comparison. And we established some rules that we would like to see in place. 

We also discussed what similiarity metrics in such an environment might look like, based on the rules. Its important to emphasie that we have really conflated the idea of semantic meaning, with distance, which is problematic, and we will need to take a deeper look at what it means to really encode something semantically. 

Finally, the Encoder Class that was introduced in the last notebook is also included in the HTM_Code.py file for us to use


<div style="background:#99ddff; color:black; padding: 10px">
<b>Add to these notes:</b>

I want to keep these notes in HTML so I don't have to host on a server, but a great exceriise is to use the ipython widgets to experience this like it happens in the video. 
</div>

In [14]:
IFrame("https://www.youtube.com/embed/PTYlge2K1G8", width=600, height=300)

This episode is about how we might build Date and Time Encoder, but the way I like to think of this is . But the idea behind this is really that join multiple encoders togehter. For example, I might be interested in an encoder that takes 365 values, one for each day of the year, or just 2 values (one for whether it is weekend or not the weekend), 4 values (telling me which of the seasons of the year it is), or a minute encoder (telling which minute of the day). 

Under the hood, these are each no different from the encoders that we were already looking at. Each value that would stored is simply and SDR with active bits and an SDR size. 

I could join each of these encoders together (by simply concatentating the arrays), and I would in principle have an encoder that tracks the meaning of theach these, and I could feed into it a date and time, and see thier differen they are. 

To explore this further, let's build another class: a multi enncoder

To explore this, let's use some data. There is some really interesting data that will turn up in episode's 7 and 8 in the context of the Spatial Pooler, that has some interesting info, but for now, let's use Baltimore Crime Data. This data has nice coverage across a number of data points, descriptive names, some categorical variables, the footprint isn't too big but it gives us a nice sample of 96k records

Information available <a href="https://data.baltimorecity.gov/datasets/baltimore::part1-crime-2015-to-2016/about">https://data.baltimorecity.gov/datasets/baltimore::part1-crime-2015-to-2016/about</a>


In [15]:
df = pd.read_csv("./data/Part1_Crime_2015_to__2016.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96976 entries, 0 to 96975
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowID            96976 non-null  int64  
 1   CrimeDateTime    96976 non-null  object 
 2   CrimeTime        96969 non-null  object 
 3   CrimeCode        96976 non-null  object 
 4   Location         96399 non-null  object 
 5   Description      96976 non-null  object 
 6   Inside_Outside   94136 non-null  object 
 7   Weapon           19832 non-null  object 
 8   Post             96797 non-null  object 
 9   District         96797 non-null  object 
 10  Neighborhood     96790 non-null  object 
 11  Latitude         96921 non-null  float64
 12  Longitude        96921 non-null  float64
 13  GeoLocation      96976 non-null  object 
 14  Premise          94007 non-null  object 
 15  VRIName          11491 non-null  object 
 16  Total_Incidents  96976 non-null  int64  
 17  HashedRecord

The first thing I want to do is fix some formatting and some derived data, so we can see day, season which will show us more about our encoder, and how we can use it.

In [16]:
df = pd.read_csv("./data/Part1_Crime_2015_to__2016.csv")
df.CrimeDateTime = df.CrimeDateTime.str.slice(0, -8)
df.CrimeDateTime= pd.to_datetime(df.CrimeDateTime)
df['weekdayCodeWhenEventReported'] = [d.weekday() for d in df.CrimeDateTime]
df['monthCodeWhenEventReported'] = df['CrimeDateTime'].dt.month
df['seasonCodeWhenEventReported'] = (df['CrimeDateTime'].dt.month - 1) % 4
df['isWeekend'] = np.where(df.weekdayCodeWhenEventReported > 4, True, False)
df = df.drop('VRIName', axis=1)
df = df.drop('HashedRecord', axis=1)
df = df.drop('ObjectId', axis=1)

In [17]:
df.sample(n = 5).transpose()

,33486,71179,34397,49022,47343
RowID,216541,254200,217668,232213,230550
CrimeDateTime,2016-05-14 00:00:00,2015-07-25 00:00:00,2016-05-05 00:00:00,2015-12-30 00:00:00,2016-01-13 00:00:00
CrimeTime,0:05:00,22:47:00,15:00:00,6:41:00,3:00:00
CrimeCode,6B,4E,6J,4D,4E
Location,1800 N WOODYEAR ST,1500 RUSSELL ST,1600 KELLY AVE,1800 W LEXINGTON ST,2100 E MONUMENT ST
Description,LARCENY,COMMON ASSAULT,LARCENY,AGG. ASSAULT,COMMON ASSAULT
Inside_Outside,O,O,NaN,I,O
Weapon,NaN,NaN,NaN,HANDS,NaN
Post,734,941,533,711,321
District,WESTERN,SOUTHERN,NORTHERN,WESTERN,EASTERN


There is also not really a sense of a Scalar Number in this data, and it would nice to see how that plays out in an encoder which I want to use, so for our purposes I also am going to create complete fictional variable which I will call 'investigativeResourcesRequired', and I will make that correlated to description

In [18]:
descriptionValues = pd.DataFrame(pd.value_counts(df.Description)).index.values
startingVal = 20
intervalVal = 5
sdForDistribution = 4

frameList = []

for i in range(len(descriptionValues)):
    startingVal = startingVal + intervalVal
    numberOfValuesToCreate = len(df[df.Description == descriptionValues[i]])
    randomDistributedValues = np.random.normal(startingVal, sdForDistribution, numberOfValuesToCreate)
    newFrame = df[df.Description == descriptionValues[i]]
    newFrame = newFrame.reset_index()
    newFrame.drop('index', axis=1)
    newFrame['investigativeResourcesRequired'] = randomDistributedValues
    frameList.append(newFrame)

df = pd.concat(frameList, ignore_index=True)
df = df.drop('index', axis=1)

Let's have a look at a sample of our final dataset

In [19]:
df.sample(n=5).transpose()

,37378,20225,75675,61981,9613
RowID,189598,276804,197115,262176,227759
CrimeDateTime,2016-11-16 00:00:00,2015-01-31 00:00:00,2016-09-28 00:00:00,2015-06-02 00:00:00,2016-02-10 00:00:00
CrimeTime,11:15:00,22:00:00,13:00:00,8:00:00,10:00:00
CrimeCode,4E,6E,7A,6D,6G
Location,4400 KAVON AVE,1800 LETITIA AVE,4000 REXMERE RD,6300 EASTERN AVE,0 N LAKEWOOD AVE
Description,COMMON ASSAULT,LARCENY,AUTO THEFT,LARCENY FROM AUTO,LARCENY
Inside_Outside,O,O,O,I,I
Weapon,NaN,NaN,NaN,NaN,NaN
Post,442,831,412,232,222
District,NORTHEAST,SOUTHWEST,NORTHEAST,SOUTHEAST,SOUTHEAST


What we are left with is a bunch of differnt types of records: some numerical records on a range, as well as some categorical records. There is some interesting data here. I will leave it to you to explore, but the point is it raises some interesting challenges to what we mean by semantic similarity, and how we would put this in some encoder that captures the information as a whole 

Let's create 4 encoders from this data: 
- a timeOfDay encoder 
- a dayOfWeekEncoder 
- an isWeekend encoder
- a season encoder and an 

We will set a global size for each 1, and 8 values

In [22]:
bit_space_size_choice = 30
number_of_bits_used_to_encode_value_choice = 6
e1 = hc.Encoder(bit_space_size = bit_space_size_choice,
                number_of_bits_used_to_encode_value = number_of_bits_used_to_encode_value_choice,
                min_val = 0,
                max_val = 91,
             resolution=2,
            is_randomly_distributed = False,
            clip_values_outside_range = True,
            is_periodic = True)
e1.get_summary()
e1.encode_value_in_bit_space(0)
e1.encode_value_in_bit_space(1)
e1.encode_value_in_bit_space(2)
e1.encode_value_in_bit_space(3)
e1.encode_value_in_bit_space(4)
e1.encode_value_in_bit_space(5)
e1.encode_value_in_bit_space(90)

e1.get_summary()
e1.buckets
#e1.encoded_values_bit_locations
e1.updateEncodedValuesDictionary()

----------------- SUMMARY -------------------------
|L3| Bit Space Size:  30
|L4| Number of bits to be used when encoding each value: 6
|L5| Range of values that can be encoded: From  0  to  91
|L6| Number of buckets available in bit space: Unlimited (is periodic)
|L1| Encode periodically:  True
|L1| Values are encoded as are randomly distributed arrays:  False
|L1| Resolution:  2
|L1| Similiarity between encodings:  1
|L2| Values outside range will to be clipped:  True
|L7| Encoded values bit locations:
  []
----------------------------------------------------

Encoding the value -> 0

Encoding the value -> 0

Encoding the value -> 1

Encoding the value -> 1

Encoding the value -> 2

Encoding the value -> 2

Encoding the value -> 45
----------------- SUMMARY -------------------------
|L3| Bit Space Size:  30
|L4| Number of bits to be used when encoding each value: 6
|L5| Range of values that can be encoded: From  0  to  91
|L6| Number of buckets available in bit space: Unlimited (is p

In [23]:
e1.encodedValues

{'0': {'bitLocations': [0, 1, 2, 3, 4, 5], 'encodedValues': array([0, 1])},
 '1': {'bitLocations': [1, 2, 3, 4, 5, 6], 'encodedValues': array([2, 3])},
 '2': {'bitLocations': [2, 3, 4, 5, 6, 7], 'encodedValues': array([4, 5])},
 '45': {'bitLocations': [15, 16, 17, 18, 19, 20],
  'encodedValues': array([90, 91])}}

Now onto next episode of HTM School

In [11]:
class MultiEncoder:
    def __init__(self):
        self.encoders = []
        self.bit_space_size = None
    def add_encoder(self, encoder):
        self.encoders.append(encoder)
    def join_encoders(self):
        pass

So this is a really powerful idea. We can think about multiple time series unfolding, date time components. Or we could think of symbolic music, track key changes

In [12]:
def compute_union_and_overlap(SDR1_on_bits, SDR2_on_bits):
    union = list(set(SDR1_on_bits).union(SDR2_on_bits))
    overlap = list(set(SDR1_on_bits).intersection(SDR2_on_bits))
    
    return({"union": union, "overlap": overlap})



This is starting to seem more like we saw in the early notebooks, we can how semantic similairty is affected by noise and subsampling

There is alot more we can do with encoders. Delta encoder, log encoder. A geospatial encoder particularly interesting, enconding values on a sphere, what other geometrical shapes, opens us up to diffent types of geometry adn topology also

Let's look at more encoders: 
https://numenta.com/assets/pdf/biological-and-machine-intelligence/BaMI-Encoders.pdf

IMportant to capture semantics properly

4 principles recall..... eg consider two numbers should be semantically similiar 

Note for encoders, we can encode anything we can put in a relationship of 

we need encoders to have to to incorporate noise and subsamplinig 

Encoding Daa for HTM systems - Purdy


Date Time Encoder

 
